In [31]:
from tweepy.streaming import StreamListener
from tweepy.auth import OAuthHandler
from tweepy import Stream

from datetime import datetime
import json
import time

import pandas as pd
from pymongo import MongoClient
import pymongo

In [2]:
#pyenv
import os
from dotenv import load_dotenv
load_dotenv()
CONSUMER_KEY = os.environ.get('CONSUMER_KEY')
CONSUMER_SECRET = os.environ.get('CONSUMER_SECRET')
ACCESS_TOKEN = os.environ.get('ACCESS_TOKEN')
ACCESS_TOKEN_SECRET = os.environ.get('ACCESS_TOKEN_SECRET')

In [3]:
auth = OAuthHandler(CONSUMER_KEY, CONSUMER_SECRET)
auth.set_access_token(ACCESS_TOKEN, ACCESS_TOKEN_SECRET)
#api = tweepy.API(auth)
#api.update_status('tweepy + oauth!')

In [4]:
client = MongoClient('localhost', 27017)
db = client.twitterdb
col = db.tweets
raw_col = db.raw_data
keywords = ['bieber']

In [66]:
class MyListener(StreamListener):
    def __init__(self):
        super().__init__()
        self.counter = 0
        self.limit = 100
        #stopwatch(30)
        
    def on_data(self,data):
        
        tweet = json.loads(data)
        print(tweet)
        """   
        For each message, we will need the following:

            The message ID
            The creation date of the message as epoch value
            The text of the message
            The author of the message

        For each author, we will need the following:

            The user ID
            The creation date of the user as epoch value
            The name of the user
            The screen name of the user
        """

        created_at = tweet["created_at"]
        id_str = tweet["id_str"]
        text = tweet["text"]
        
        user_id = tweet["user"]["id_str"]
        user_name = tweet["user"]["name"]
        screen_name = tweet["user"]["screen_name"]
        user_created_at = tweet["user"]["created_at"]
        
        
        obj = {
            "message_created_at":time.mktime(time.strptime( created_at,"%a %b %d %H:%M:%S +0000 %Y" )),
            "message_id":id_str,
            "message_text":text,
            "author_user_id": user_id,
            "author_creation_date": time.mktime(time.strptime( user_created_at,"%a %b %d %H:%M:%S +0000 %Y" )),
            "author_user_name": user_name,
            "author_screen_name": screen_name
        }
        #print(obj)
        print('====================')
        tweetind = col.insert_one(obj).inserted_id
        #print(data)
        tweetind_2 = raw_col.insert_one( json.loads(json.dumps(tweet))  ).inserted_id
        return True
    
    def on_error(self, status):
        print(status)
        return False
    
    def on_status(self, status):
        try:
            userid = status.user.id_str
            geo = str(status.coordinates)
            if geo != "None":
                print(userid + ',' + geo)
            else:
                print("No coordinates")
            self.counter += 1
            if self.counter < self.limit:
                return True
            else:
                twitterStream.disconnect()
        except BaseException as e:
            print('failed on_status,',str(e))
            time.sleep(5)

In [67]:
mylistener = MyListener()
mystream = Stream(auth, listener = mylistener)

In [68]:
print('start stream')
#mystream.filter(track=keywords)
mystream.filter(track = keywords)
print('close stream')

start stream
{'created_at': 'Sun May 05 14:32:36 +0000 2019', 'id': 1125045637094023169, 'id_str': '1125045637094023169', 'text': 'RT @svvinte: you know what is also not okay and actually killing people\n\ndomestic abuse https://t.co/bD5Hw7nmHa', 'source': '<a href="http://twitter.com/download/iphone" rel="nofollow">Twitter for iPhone</a>', 'truncated': False, 'in_reply_to_status_id': None, 'in_reply_to_status_id_str': None, 'in_reply_to_user_id': None, 'in_reply_to_user_id_str': None, 'in_reply_to_screen_name': None, 'user': {'id': 885036420880101376, 'id_str': '885036420880101376', 'name': 'Falopa Queer ☭', 'screen_name': 'aseldelcap', 'location': 'Montevideo, Uruguay', 'url': 'http://instagram.com/axelhoying', 'description': 'blanco e indígena// Antes muerta que sencilla. Bioquimicx en desarrollo. Schrödinger is my daddy 😌✨// Bloqueo gente que no banco', 'translator_type': 'none', 'protected': False, 'verified': False, 'followers_count': 319, 'friends_count': 296, 'listed_count': 2,

{'created_at': 'Sun May 05 14:32:37 +0000 2019', 'id': 1125045638859718657, 'id_str': '1125045638859718657', 'text': 'RT @G0ALSL4RRY: Así estoy anotando los nombres de todos los que están defendiendo a Chris Brown o están de acuerdo con la publicación de Ju…', 'source': '<a href="http://twitter.com/download/iphone" rel="nofollow">Twitter for iPhone</a>', 'truncated': False, 'in_reply_to_status_id': None, 'in_reply_to_status_id_str': None, 'in_reply_to_user_id': None, 'in_reply_to_user_id_str': None, 'in_reply_to_screen_name': None, 'user': {'id': 535650688, 'id_str': '535650688', 'name': 'Miss Honolulu', 'screen_name': 'O_Holberg', 'location': 'Madrid, España', 'url': None, 'description': '21. Blood type: matte black with a hint of gold // ♡ ♡ Beyoncé. Empire state of mind ♡ ♡ Que la Tierra te sea leve.', 'translator_type': 'none', 'protected': False, 'verified': False, 'followers_count': 243, 'friends_count': 181, 'listed_count': 7, 'favourites_count': 6892, 'statuses_count': 18601, '

{'created_at': 'Sun May 05 14:32:37 +0000 2019', 'id': 1125045639828656129, 'id_str': '1125045639828656129', 'text': 'RT @nervstyler: FAMOSOS QUE HAN COMENTADO "A FAVOR" EN LA FOTO DE JUSTIN BIEBER https://t.co/BcrL9NSVHD', 'source': '<a href="http://twitter.com/download/android" rel="nofollow">Twitter for Android</a>', 'truncated': False, 'in_reply_to_status_id': None, 'in_reply_to_status_id_str': None, 'in_reply_to_user_id': None, 'in_reply_to_user_id_str': None, 'in_reply_to_screen_name': None, 'user': {'id': 2212175950, 'id_str': '2212175950', 'name': 'Flawless ♀️🌻', 'screen_name': 'larq_11', 'location': 'Hogwarts ', 'url': None, 'description': "Hufflepuff 💛 / Teen Wolf / Shadowhunters / Liars / Sense8 / Skam / Grey's anatomy / Merlí / The Vamps💖 / Froy💚 / OT2017💞", 'translator_type': 'none', 'protected': False, 'verified': False, 'followers_count': 496, 'friends_count': 978, 'listed_count': 2, 'favourites_count': 87383, 'statuses_count': 24325, 'created_at': 'Sat Dec 07 13:09:37 +

{'created_at': 'Sun May 05 14:32:37 +0000 2019', 'id': 1125045641162514433, 'id_str': '1125045641162514433', 'text': 'RT @fixedbailey: famosos que concordam com o justin bieber em relação ao chris brown; a thread. https://t.co/WudNHs972u', 'source': '<a href="https://mobile.twitter.com" rel="nofollow">Twitter Web App</a>', 'truncated': False, 'in_reply_to_status_id': None, 'in_reply_to_status_id_str': None, 'in_reply_to_user_id': None, 'in_reply_to_user_id_str': None, 'in_reply_to_screen_name': None, 'user': {'id': 1069730125984489477, 'id_str': '1069730125984489477', 'name': 'ʟısıѧ ; ᥲs ᥕιᥒtᥱr ❄', 'screen_name': '_moonrisee', 'location': 'Wakanda', 'url': None, 'description': "-\xa0𝔀𝓱𝓸'𝓼\xa0𝓰𝓸𝓷𝓷𝓪\xa0𝓬𝓪𝓽𝓬𝓱\xa0𝓶𝓮\xa0𝔀𝓱𝓮𝓷\xa0𝓲\xa0𝓯𝓪𝓵𝓵 \xa0-", 'translator_type': 'none', 'protected': False, 'verified': False, 'followers_count': 727, 'friends_count': 768, 'listed_count': 3, 'favourites_count': 10387, 'statuses_count': 12652, 'created_at': 'Mon Dec 03 23:08:31 +0000 2018', 'utc_offset': None

{'created_at': 'Sun May 05 14:32:38 +0000 2019', 'id': 1125045642265550848, 'id_str': '1125045642265550848', 'text': 'No', 'source': '<a href="http://twitter.com/download/iphone" rel="nofollow">Twitter for iPhone</a>', 'truncated': False, 'in_reply_to_status_id': None, 'in_reply_to_status_id_str': None, 'in_reply_to_user_id': None, 'in_reply_to_user_id_str': None, 'in_reply_to_screen_name': None, 'user': {'id': 107009668, 'id_str': '107009668', 'name': 'Lau', 'screen_name': 'LauraWasiWasi', 'location': 'Tenerife', 'url': None, 'description': 'Dile los rollos patri ~ 16demarzo ~ 25denoviembre ♥️', 'translator_type': 'regular', 'protected': False, 'verified': False, 'followers_count': 2064, 'friends_count': 803, 'listed_count': 19, 'favourites_count': 13051, 'statuses_count': 32987, 'created_at': 'Thu Jan 21 09:36:25 +0000 2010', 'utc_offset': None, 'time_zone': None, 'geo_enabled': True, 'lang': 'es', 'contributors_enabled': False, 'is_translator': False, 'profile_background_color': 'FF

{'created_at': 'Sun May 05 14:32:38 +0000 2019', 'id': 1125045644836655104, 'id_str': '1125045644836655104', 'text': 'RT @gomezsglowing: THIS is what Chris Brown did to Rihanna. don’t EVER ever call it a mistake. abusing women is not a fuck!ng “mistake”. Wo…', 'source': '<a href="http://twitter.com/download/iphone" rel="nofollow">Twitter for iPhone</a>', 'truncated': False, 'in_reply_to_status_id': None, 'in_reply_to_status_id_str': None, 'in_reply_to_user_id': None, 'in_reply_to_user_id_str': None, 'in_reply_to_screen_name': None, 'user': {'id': 1123324790016757761, 'id_str': '1123324790016757761', 'name': '-A', 'screen_name': 'stargasemyg', 'location': None, 'url': None, 'description': 'ℑ 𝔬𝔫𝔩𝔶 𝔩𝔲𝔰𝔱 𝔣𝔬𝔯 𝔱𝔥𝔦𝔫𝔤𝔰 𝔞𝔫𝔡 𝔭𝔢𝔬𝔭𝔩𝔢 ℑ 𝔰𝔢𝔢 𝔭𝔬𝔱𝔢𝔫𝔱𝔦𝔞𝔩 𝔦𝔫', 'translator_type': 'none', 'protected': False, 'verified': False, 'followers_count': 18, 'friends_count': 51, 'listed_count': 0, 'favourites_count': 1207, 'statuses_count': 262, 'created_at': 'Tue Apr 30 20:34:35 +0000 2019', 'utc_offset': None, '

{'created_at': 'Sun May 05 14:32:39 +0000 2019', 'id': 1125045646220828672, 'id_str': '1125045646220828672', 'text': 'RT @tayIucifer97: y’all are wasting time dragging taylor swift because of a couple of drums while male artists are getting away with sh*t l…', 'source': '<a href="http://twitter.com/download/iphone" rel="nofollow">Twitter for iPhone</a>', 'truncated': False, 'in_reply_to_status_id': None, 'in_reply_to_status_id_str': None, 'in_reply_to_user_id': None, 'in_reply_to_user_id_str': None, 'in_reply_to_screen_name': None, 'user': {'id': 516036984, 'id_str': '516036984', 'name': 'luíza', 'screen_name': 'prfctaylor', 'location': 'ǝʌɐ uooɯʎǝuoɥ', 'url': 'https://twitter.com/tayloreput/status/997707117871190016?s=21', 'description': 'fan account', 'translator_type': 'regular', 'protected': False, 'verified': False, 'followers_count': 7003, 'friends_count': 5684, 'listed_count': 62, 'favourites_count': 15802, 'statuses_count': 80657, 'created_at': 'Tue Mar 06 00:55:07 +0000 2012',

KeyboardInterrupt: 

In [53]:
all_values = col.find().sort( "message_created_at" , pymongo.ASCENDING )
#.sort([("field1",pymongo.ASCENDING), ("field2",pymongo.DESCENDING)])

for x in all_values:
    print(x)

{'_id': ObjectId('5cceefb48eca76e4ed12e6d7'), 'message_created_at': 1557076444.0, 'message_id': '1125040973535272960', 'message_text': 'RT @tayIucifer97: y’all are wasting time dragging taylor swift because of a couple of drums while male artists are getting away with sh*t l…', 'author_user_id': '', 'author_creation_date': '', 'author_user_name': '', 'author_screen_name': ''}
{'_id': ObjectId('5cceefb48eca76e4ed12e6d8'), 'message_created_at': 1557076447.0, 'message_id': '1125040982456774657', 'message_text': 'RT @paulinagdavalos: ladies we’re moving on to 🛫 bad bunny https://t.co/uxykj6yZFV', 'author_user_id': '', 'author_creation_date': '', 'author_user_name': '', 'author_screen_name': ''}
{'_id': ObjectId('5cceefb48eca76e4ed12e6d9'), 'message_created_at': 1557076447.0, 'message_id': '1125040982771339265', 'message_text': 'RT @whomath: espancar/abusar de mulheres também mata, justin https://t.co/XOxyKqq1bZ', 'author_user_id': '', 'author_creation_date': '', 'author_user_name': '', 'au

In [69]:
import json
import csv
from bson.json_util import dumps
for i in all_values:
    print(i)
#json_values = all_values.toArray()
j = json.loads(dumps(col.find().sort( "created_at" , pymongo.ASCENDING )))
#print(dumps(col.find().sort( "created_at" , pymongo.ASCENDING )))
#for element in j:
#    print(element)

with open('output.tsv', 'w') as output_file:
    dw = csv.DictWriter(output_file, sorted(j[0].keys()), delimiter='\t')
    dw.writeheader()
    dw.writerows(j)

In [87]:
from datetime import datetime, time
now = datetime.now()
#beginning_of_day = datetime.combine(now.date(), time(0))

while (now - beginning_of_day).seconds < 46750:
    now = datetime.now()    
    os.system('cls')
    print ( (now - beginning_of_day).seconds )



46562
46562
46562
46562
46562
46562
46562
46562
46562
46562
46562
46562
46562
46562
46562
46562
46562
46562
46562
46562
46562
46562
46562
46562
46562
46562
46562
46562
46562
46562
46562
46562
46562
46562
46562
46562
46562
46562
46562
46562
46562
46562
46562
46562
46562
46562
46562
46562
46562
46562
46562
46562
46562
46562
46562
46562
46562
46562
46562
46562
46562
46562
46562
46562
46562
46562
46562
46562
46562
46562
46562
46562
46562
46562
46562
46562
46562
46562
46562
46562
46562
46562
46562
46562
46562
46562
46562
46562
46562
46562
46562
46562
46562
46562
46562
46562
46562
46562
46562
46562
46562
46562
46562
46562
46562
46562
46562
46562
46562
46562
46562
46562
46562
46562
46562
46562
46562
46562
46562
46562
46562
46562
46562
46562
46562
46562
46562
46562
46562
46562
46562
46562
46562
46562
46562
46562
46562
46562
46562
46562
46562
46562
46562
46562
46562
46562
46562
46562
46562
46562
46562
46562
46562
46562
46562
46562
46562
46562
46562
46562
46562
46562
46562
46562
46562
46562
4656

46563
46563
46563
46563
46563
46563
46563
46563
46563
46563
46563
46563
46563
46563
46563
46563
46563
46563
46563
46563
46563
46563
46563
46563
46563
46563
46563
46563
46563
46563
46563
46563
46563
46563
46563
46563
46563
46563
46563
46563
46563
46563
46563
46563
46563
46563
46563
46563
46563
46563
46563
46563
46563
46563
46563
46563
46563
46563
46563
46563
46563
46563
46563
46563
46563
46563
46563
46563
46563
46563
46563
46563
46563
46563
46563
46563
46563
46563
46563
46563
46563
46563
46563
46563
46563
46563
46563
46563
46563
46563
46563
46563
46563
46563
46563
46563
46563
46563
46563
46563
46563
46563
46563
46563
46563
46563
46563
46563
46563
46563
46563
46563
46563
46563
46563
46563
46563
46563
46563
46563
46563
46563
46563
46563
46563
46563
46563
46563
46563
46563
46563
46563
46563
46563
46563
46563
46563
46563
46563
46563
46563
46563
46563
46563
46563
46563
46563
46563
46563
46563
46563
46563
46563
46563
46563
46563
46563
46563
46563
46563
46563
46563
46563
46563
46563
46563
4656

46563
46563
46563
46563
46563
46563
46563
46563
46563
46563
46563
46563
46563
46563
46563
46563
46563
46563
46563
46563
46563
46563
46563
46563
46563
46563
46563
46563
46563
46563
46563
46563
46563
46563
46563
46563
46563
46563
46563
46563
46563
46563
46563
46563
46563
46563
46563
46563
46563
46563
46563
46563
46563
46563
46563
46563
46563
46563
46563
46563
46563
46563
46563
46563
46563
46563
46563
46563
46563
46563
46563
46563
46563
46563
46563
46563
46563
46563
46563
46563
46563
46563
46563
46563
46563
46563
46563
46563
46563
46563
46563
46563
46563
46563
46563
46563
46563
46563
46563
46563
46563
46563
46563
46563
46563
46563
46563
46563
46563
46563
46563
46563
46563
46563
46563
46563
46563
46563
46563
46563
46563
46563
46563
46563
46563
46563
46563
46563
46563
46563
46563
46563
46563
46563
46563
46563
46563
46563
46563
46563
46563
46563
46563
46563
46563
46563
46563
46563
46563
46563
46563
46563
46563
46563
46563
46563
46563
46563
46563
46563
46563
46563
46563
46563
46563
46563
4656

46563
46563
46563
46563
46563
46563
46563
46563
46563
46563
46563
46563
46563
46563
46563
46563
46563
46563
46563
46563
46563
46563
46563
46563
46563
46563
46563
46563
46563
46563
46563
46563
46563
46563
46563
46563
46563
46563
46563
46563
46563
46563
46563
46563
46563
46563
46563
46563
46563
46563
46563
46563
46563
46563
46563
46563
46563
46563
46563
46563
46563
46563
46563
46563
46563
46563
46563
46563
46563
46563
46563
46563
46563
46563
46563
46563
46563
46563
46563
46563
46563
46563
46563
46563
46563
46563
46563
46563
46563
46563
46563
46563
46563
46563
46563
46563
46563
46563
46563
46563
46563
46563
46563
46563
46563
46563
46563
46563
46563
46563
46563
46563
46563
46563
46563
46563
46563
46563
46563
46563
46563
46563
46563
46563
46563
46563
46563
46563
46563
46563
46563
46563
46563
46563
46563
46563
46563
46563
46563
46563
46563
46563
46563
46563
46563
46563
46563
46563
46563
46563
46563
46563
46563
46563
46563
46563
46563
46563
46563
46563
46563
46563
46563
46563
46563
46563
4656

46563
46563
46563
46563
46563
46563
46563
46563
46563
46563
46563
46563
46563
46563
46563
46563
46563
46563
46563
46563
46563
46563
46563
46563
46563
46563
46563
46563
46563
46563
46563
46563
46563
46563
46563
46563
46563
46563
46563
46563
46563
46563
46563
46563
46563
46563
46563
46563
46563
46563
46563
46563
46563
46563
46563
46563
46563
46563
46563
46563
46563
46563
46563
46563
46563
46563
46563
46563
46563
46563
46563
46563
46563
46563
46563
46563
46563
46563
46563
46563
46563
46563
46563
46563
46563
46563
46563
46563
46563
46563
46563
46563
46563
46563
46563
46563
46563
46563
46563
46563
46563
46563
46563
46563
46563
46563
46563
46563
46563
46563
46563
46563
46563
46563
46563
46563
46563
46563
46563
46563
46563
46563
46563
46563
46563
46563
46563
46563
46563
46563
46563
46563
46563
46563
46563
46563
46563
46563
46563
46563
46563
46563
46563
46563
46563
46563
46563
46563
46563
46563
46563
46563
46563
46563
46563
46563
46563
46563
46563
46563
46563
46563
46563
46563
46563
46563
4656

...

46574
46574
46574
46574
46574
46574
46574
46574
46574
46574
46574
46574
46574
46574
46574
46574
46574
46574
46574
46574
46574
46574
46574
46574
46574
46574
46574
46574
46574
46574
46574
46574
46574
46574
46574
46574
46574
46574
46574
46574
46574
46574
46574
46574
46574
46574
46574
46574
46574
46574
46574
46574
46574
46574
46574
46574
46574
46574
46574
46574
46574
46574
46574
46574
46574
46574
46574
46574
46574
46574
46574
46574
46574
46574
46574
46574
46574
46574
46574
46574
46574
46574
46574
46574
46574
46574
46574
46574
46574
46574
46574
46574
46574
46574
46574
46574
46574
46574
46574
46574
46574
46574
46574
46574
46574
46574
46574
46574
46574
46574
46574
46574
46574
46574
46574
46574
46574
46574
46574
46574
46574
46574
46574
46574
46574
46574
46574
46574
46574
46574
46574
46574
46574
46574
46574
46574
46574
46574
46574
46574
46574
46574
46574
46574
46574
46574
46574
46574
46574
46574
46574
46574
46574
46574
46574
46574
46574
46574
46574
46574
46574
46574
46574
46574
46574
46574
4657

46574
46574
46574
46574
46574
46574
46574
46574
46574
46574
46574
46574
46574
46574
46574
46574
46574
46574
46574
46574
46574
46574
46574
46574
46574
46574
46574
46574
46574
46574
46574
46574
46574
46574
46574
46574
46574
46574
46574
46574
46574
46574
46574
46574
46574
46574
46574
46574
46574
46574
46574
46574
46574
46574
46574
46574
46574
46574
46574
46574
46574
46574
46574
46574
46574
46574
46574
46574
46574
46574
46574
46574
46574
46574
46574
46574
46574
46574
46574
46574
46574
46574
46574
46574
46574
46574
46574
46574
46574
46574
46574
46574
46574
46574
46574
46574
46574
46574
46574
46574
46574
46574
46574
46574
46574
46574
46574
46574
46574
46574
46574
46574
46574
46574
46574
46574
46574
46574
46574
46574
46574
46574
46574
46574
46574
46574
46574
46574
46574
46574
46574
46574
46574
46574
46574
46574
46574
46574
46574
46574
46574
46574
46574
46574
46574
46574
46574
46574
46574
46574
46574
46574
46574
46574
46574
46574
46574
46574
46574
46574
46574
46574
46574
46574
46574
46574
4657

46574
46574
46574
46574
46574
46574
46574
46574
46574
46574
46574
46574
46574
46574
46574
46574
46574
46574
46574
46574
46574
46574
46574
46574
46574
46574
46574
46574
46574
46574
46574
46574
46574
46574
46574
46574
46574
46574
46574
46574
46574
46574
46574
46574
46574
46574
46574
46574
46574
46574
46574
46574
46574
46574
46574
46574
46574
46574
46574
46574
46574
46574
46574
46574
46574
46574
46574
46574
46574
46574
46574
46574
46574
46574
46574
46574
46574
46574
46574
46574
46574
46574
46574
46574
46574
46574
46574
46574
46574
46574
46574
46574
46574
46574
46574
46574
46574
46574
46574
46574
46574
46574
46574
46574
46574
46574
46574
46574
46574
46574
46574
46574
46574
46574
46574
46574
46574
46574
46574
46574
46574
46574
46574
46574
46574
46574
46574
46574
46574
46574
46574
46574
46574
46574
46574
46574
46574
46574
46574
46574
46574
46574
46574
46574
46574
46574
46574
46574
46574
46574
46574
46574
46574
46574
46574
46574
46574
46574
46574
46574
46574
46574
46574
46574
46574
46574
4657

46575
46575
46575
46575
46575
46575
46575
46575
46575
46575
46575
46575
46575
46575
46575
46575
46575
46575
46575
46575
46575
46575
46575
46575
46575
46575
46575
46575
46575
46575
46575
46575
46575
46575
46575
46575
46575
46575
46575
46575
46575
46575
46575
46575
46575
46575
46575
46575
46575
46575
46575
46575
46575
46575
46575
46575
46575
46575
46575
46575
46575
46575
46575
46575
46575
46575
46575
46575
46575
46575
46575
46575
46575
46575
46575
46575
46575
46575
46575
46575
46575
46575
46575
46575
46575
46575
46575
46575
46575
46575
46575
46575
46575
46575
46575
46575
46575
46575
46575
46575
46575
46575
46575
46575
46575
46575
46575
46575
46575
46575
46575
46575
46575
46575
46575
46575
46575
46575
46575
46575
46575
46575
46575
46575
46575
46575
46575
46575
46575
46575
46575
46575
46575
46575
46575
46575
46575
46575
46575
46575
46575
46575
46575
46575
46575
46575
46575
46575
46575
46575
46575
46575
46575
46575
46575
46575
46575
46575
46575
46575
46575
46575
46575
46575
46575
46575
4657

KeyboardInterrupt: 

In [89]:
from datetime import datetime, time

now = datetime.now()
beginning_of_day = datetime.combine(now.date(), time(0))
print ( (now - beginning_of_day).seconds)

46702


In [ ]:
import time

def stopwatch(seconds):
    start = time.time()
    time.clock()    
    elapsed = 0
    while elapsed < seconds:
        elapsed = time.time() - start
        print ("loop cycle time: %f, seconds count: %02d" % (time.clock() , elapsed)  )
        time.sleep(1)  

stopwatch(30)

In [ ]:
class LimitedListener(StreamListener):

    def __init__(self):
        super().__init__()
        self.counter = 0
        self.limit = 10

    def on_status(self, status):
        try:
            userid = status.user.id_str
            geo = str(status.coordinates)
            if geo != "None":
                print(userid + ',' + geo)
            else:
                print("No coordinates")
            self.counter += 1
            if self.counter < self.limit:
                return True
            else:
                twitterStream.disconnect()
        except BaseException as e:
            print('failed on_status,',str(e))
            time.sleep(5)

In [ ]:
limited_listener = LimitedListener()
limited_stream = Stream(auth, listener = limited_listener)
print('inicia limited stream')
limited_stream.filter(track=keywords)
print('fecha limited stream')

In [ ]:
"""
client = MongoClient('localhost', 27017)
db_test = client.testdb
collection_test = db_test.col_test
obj_test = {"putetinho": "ayn ayn"}
collection_test.insert_one(obj_test)
"""

In [ ]:
col.find_one()

In [ ]:
dataset = [{"created_at":item["created_at"],
           "text":item["text"],
           } for item in col.find()]

In [ ]:
df = pd.DataFrame(dataset)

In [ ]:
df.head()

In [ ]:
cv = CountVectorizer()
count_matrix = cv.fit_transform(df.text)

In [ ]:
word_count = pd.DataFrame(cv.get_feature_names(), columns=["word"])
word_count["count"] = count_matrix.sum(axis=0).tolist()[0]
word_count = word_count.sort_values("count", ascending=False).reset_index(drop=True)
word_count[:50]